In [3]:
%matplotlib inline

from keras.datasets import mnist

import tensorflow as tf
import numpy as np
import os

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from matplotlib import pyplot as plt

from tensorflow.keras.layers import Dense, Flatten, Conv2D, BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose, Reshape, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from PIL import Image

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.8.0


In [6]:
data_path = r"C:\Users\Alberto Parenti\Downloads\STUDY\NOVA IMS\DEEP LEARNING\PROJECT\crop_part1"

#We need a way to convert our training set into the correct format

def load_real_samples(image_path):
    
    #Defining master array
    master_list = list()

    for image in os.listdir(image_path):
        #loading image
        img = load_img(os.path.join(image_path,image))
        # convert to numpy array
        img_array = img_to_array(img)

        #img_array = tf.image.resize(img_array, [28, 28])
        #Standardizing to float
        img_array = img_array.astype("float32")
        #Getting value between 0 and 1
        img_array/=255.0

        master_list.append(img_array)

    return np.array(master_list)

array_data = load_real_samples(data_path)

In [8]:
array_data.shape

(125, 200, 200, 3)

In [9]:
#Defining image shape

img_rows = 200
img_cols = 200
channels = 3

img_shape = (img_rows, img_cols, channels)

In [10]:
def build_generator():
    noise_shape = (100,) #Defining the latent vector of size 100

    #Defining our model with 4 dense layers of increasing dimension
    model = Sequential()

    model.add(Dense(256, input_shape = noise_shape))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    model.add(Dense(2048))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum = 0.8))

    model.add(Dense(np.prod(img_shape), activation = "tanh"))
    model.add(Reshape(img_shape))

    model.summary()

    #Transforming the latent space vector into an tf input
    noise = Input(shape = noise_shape)
    #Feeding that latent space into the model to create an image
    img = model(noise) #Generated image

    return Model(noise, img)



In [11]:
#Building a discriminator to guess at whether an image is real or not
#Model has 4 layers of decreasing dimension

def build_discriminator():

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(1024))

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))

    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    #Converting the image into a tf input and passing that to the model
    img = Input(shape=img_shape)
    #Retrieving the output prediction from the model
    validity = model(img)

    return Model(img, validity) #The validity is whether or not the model thinks the image is real
    

In [12]:
def train(dataset, epochs, batch_size=128, save_interval=500):
    
    half_batch = int(batch_size / 2)

    disc_accuracy_list = []
    disc_loss_list = []
    generator_loss_list = []
    epoch_list = []

    for epoch in range(epochs):
        
        # ---------------------
        #  Discriminator Training
        # ---------------------

        # Select a random half batch of real images from our array data stored in array_data

        #Storing the indexes of those images in idx
        indx = np.random.randint(0, array_data.shape[0], half_batch)

        #Retreiving those indexes from the array
        imgs = array_data[indx]

        #Generating a half batch worth of latent space vectors which will comprise half of what
        #we feed to the discriminator
        latent_space = np.random.normal(0, 1, (half_batch, 100))
        
        # Generate a half batch of fake images from the half batch of latent spaces
        generated_imgs = generator.predict(latent_space)

        # Training the discriminator on real and fake images but not at the same time, always holding one constant
        #first on the real
        disc_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))

        #then on the fake generated images
        disc_loss_fake = discriminator.train_on_batch(generated_imgs, np.zeros((half_batch, 1)))

        #take average loss from real and fake images. 
        disc_loss = 0.5 * np.add(disc_loss_real, disc_loss_fake) 

        #Now within the same for loop we train our Generator model by setting the input latent_space and
        #ultimately training the Generator to have the Discriminator label its samples as valid or false.
        
        # ---------------------
        #  Train Generator
        # ---------------------

        #Create latent_space vectors as input for generator. 
        #Create as many latent_space vectors as defined by the batch size. 
        latent_space = np.random.normal(0, 1, (batch_size, 100)) 

        #Creating a vector of ones in order to trick the discriminator into thinking that the image is real
        valid_y = np.array([1] * batch_size) #Creates an array of all ones of size=batch size

        # Generator is part of the combined model where it paired with the discriminator
        # Train the generator with latent_space as x and 1 as y. 
        generator_loss = combined.train_on_batch(latent_space, valid_y)


        #In order for us to keep track of the epochs elapsed and the results we print the following
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, disc_loss[0], 100*disc_loss[1], generator_loss))

        #Appending the loss and accuracy metrics to lists
        epoch_list.append(epoch)
        disc_loss_list.append(disc_loss[0])
        disc_accuracy_list.append(100*disc_loss[1])
        generator_loss_list.append(generator_loss)

        # If we are at the specified interval save generated image of samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

    #At the end of the epochs plot the discriminator loss per epoch
    plt.plot(epoch_list, disc_loss_list)
    plt.show()

def save_imgs(epoch):
    r, c = 5, 5
    latent_space = np.random.normal(0, 1, (r * c, 100))
    generated_imgs = generator.predict(latent_space)

    # Scaling our image
    generated_imgs = 0.5 * generated_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(generated_imgs[cnt, :,:,:])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/generation_%d.png" % epoch)
    plt.close()

In [15]:
optimizer = Adam(0.0001, 0.5)  #Learning rate and momentum.

# We are first going to build the discriminator and then train the generator as part of the combined model later
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

#Building and compiling our Generator, picking the loss function and optimizer
#Since we are generating fake images there is no need to track any metrics.
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

#Creating first the input latent space and then defining the generator  
z = Input(shape=(100,))   #Our random vector to be used as an input

#Storing the image as img
img = generator(z)

#Ensuring that when we combine Discriminator and Generator networks we only train the latter 
#This is to ensure that while the generator is being trained, the weights of the discriminator are not being adjusted
#Note that this has no impact on the discriminator training above    
discriminator.trainable = False  

#Here our generator takes our image and classifies it as either real or fake  
r_or_f = discriminator(img) 


#Here we combine the gen and disc models and define our loss function and optimizer. 
#To be sure, we are only training the generator here-
#The objective in this step is for the generator to fool the discriminator  
# The final combined model  (which is a stacked generator and discriminator) takes
# noise (latent space) as an input => generates images => determines the validity of those images
combined = Model(z, r_or_f)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

#Setting the model with the needed params
train(array_data,epochs=150, batch_size=128, save_interval=500)

#Saving model for future use
generator.save('image_generator_model.h5')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 120000)            0         
                                                                 
 dense_18 (Dense)            (None, 1024)              122881024 
                                                                 
 flatten_5 (Flatten)         (None, 1024)              0         
                                                                 
 dense_19 (Dense)            (None, 512)               524800    
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_20 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 256)              